# Recurrent Neural Networks
## Covers
   > Character level Vanilla RNN implementation.     
    Generating a sequence from a random char picked from the test set.  
    Experiments to establish the roles the state variables play.
   
 <img src='assets/rnn.png'>


# Vanilla implementation

In [10]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
print('Python version:',sys.version)
print('TF version:',tf.__version__)
np.random.seed(42)
tf.set_random_seed(42)
SEED = 42

Python version: 3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]
TF version: 1.8.0


<img src='assets/input.png'>

> The above toy example was chosen as input to check whether the network can figure out whether the singer is a he/she based on the context. The n/w should remember if it has seen either bob or julia before and it predicts 'he sings' or 'she sings' appropriately. 

In [2]:
#Create a test data
#Change the input data to play around
text = 'Bob is a singer, he sings well. Julia is also a singer, she sings better '
x = list(text[0:len(text)-1])

#y is the next char of x
y = list(text[1:len(text)])

In [3]:
#Converting the input to one hot encoding 
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

#String to int
le = LabelEncoder()
vocab = list(set(text))
le.fit(vocab)
text_int = le.transform(vocab)
print('**Input :x \n', x)
x = le.transform(x)
y = le.transform(y)
print('\n**LabelEncoder :x \n', x)
#One-hot encoding
lb = LabelBinarizer()
lb.fit(text_int)
x = lb.transform(x)
y = lb.transform(y)
print('\n**LabelBinarizer :x \n', x)
print('\n**Transform back :x \n', le.inverse_transform(lb.inverse_transform(x)))
vocab_dict = dict(zip(text_int.tolist(), vocab))
vocab_reverse_dict = dict(zip(vocab, text_int.tolist()))
vocab_onehot = lb.transform(text_int)
vocab_onehot_dict = dict(zip(text_int.tolist(), vocab_onehot))


**Input :x 
 ['B', 'o', 'b', ' ', 'i', 's', ' ', 'a', ' ', 's', 'i', 'n', 'g', 'e', 'r', ',', ' ', 'h', 'e', ' ', 's', 'i', 'n', 'g', 's', ' ', 'w', 'e', 'l', 'l', '.', ' ', 'J', 'u', 'l', 'i', 'a', ' ', 'i', 's', ' ', 'a', 'l', 's', 'o', ' ', 'a', ' ', 's', 'i', 'n', 'g', 'e', 'r', ',', ' ', 's', 'h', 'e', ' ', 's', 'i', 'n', 'g', 's', ' ', 'b', 'e', 't', 't', 'e', 'r']

**LabelEncoder :x 
 [ 3 13  6  0 10 15  0  5  0 15 10 12  8  7 14  1  0  9  7  0 15 10 12  8
 15  0 18  7 11 11  2  0  4 17 11 10  5  0 10 15  0  5 11 15 13  0  5  0
 15 10 12  8  7 14  1  0 15  9  7  0 15 10 12  8 15  0  6  7 16 16  7 14]

**LabelBinarizer :x 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

**Transform back :x 
 ['B' 'o' 'b' ' ' 'i' 's' ' ' 'a' ' ' 's' 'i' 'n' 'g' 'e' 'r' ',' ' ' 'h'
 'e' ' ' 's' 'i' 'n' 'g' 's' ' ' 'w' 'e' 'l' 'l' '.' ' ' 'J' 'u' 'l' 'i'
 'a' ' ' 'i' 's' ' ' 'a' 'l' 's' 'o' ' ' 'a' ' ' 's' 'i' 'n' 'g' 'e' 'r'


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# Building a char RNN

 >The RNN has number of states = 'n_state_size' and the number hidden units = hidden_size   
 The inputs X, Y and fed in the size = {length of each one_hot_encoded input, no of states}    
 The RNN class has the following methods:   
     * init = Initializes the weights, state variables, loss function and optimizers. Creates the state graph.    
     * train = Trains the classifier given the inputs X, Y and the 'valid_char' string for testing during training
     * test = generates a sequence based on a input test string (of length state_size)
 

In [4]:

class charRNN(object):
    
    def __init__(self, n_hidden, n_vocab_size, n_state_size, vocab_dict, vocab_reverse_dict, vocab_onehot_dict):
        
        self.hidden_size = n_hidden
        self.state_size = n_state_size
        self.truncated_backprop_length = n_state_size
        self.vocab_size = n_vocab_size
        
        #copy the dictionary
        self.vocab_onehot_dict = vocab_onehot_dict
        self.vocab_reverse_dict = vocab_reverse_dict
        self.vocab_dict = vocab_dict
               
        # model inputs
        self.x = tf.placeholder(tf.float32, [self.vocab_size, self.truncated_backprop_length])
        self.y = tf.placeholder(tf.int32, [self.vocab_size, self.truncated_backprop_length])
        
        inputs_series = tf.unstack(self.x, axis=1) # unpack the columns
        labels_series = tf.unstack(self.y, axis=1) # unpack the columns
        
        # get the network weights
        network_weights = self._initialize_weights()
        self.weights = network_weights
        
        # model
        current_state = self.weights['init_state']
        states_series = []
        for X_i in inputs_series:
            X_i = tf.reshape(X_i, [-1, self.vocab_size])       
            next_state = tf.tanh(tf.matmul(X_i, self.weights['Wxh']) + 
                                 tf.matmul(current_state, self.weights['Whh']) + self.weights['bhh'])           
            states_series.append(next_state)
            current_state = next_state
        self.states_series = states_series
        
        #logits and cost
        logits_series = [tf.matmul(state, self.weights['Why'])+ self.weights['bhy'] for state in self.states_series]
        
        self.pred_series = [tf.nn.softmax(logits) for logits in logits_series]
        self.loss = tf.reduce_mean([tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, labels in zip(logits_series, labels_series)])
        self.optimizer = tf.train.AdagradOptimizer(0.05).minimize(self.loss)
        
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
    
        
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['init_state'] = tf.Variable(tf.zeros([1, self.hidden_size]), dtype=tf.float32, name='init_state')
        all_weights['Wxh'] = tf.Variable(tf.truncated_normal((self.vocab_size, self.hidden_size), stddev=0.01, seed=SEED), 
                                         dtype=tf.float32, name='Wxh')
        all_weights['Whh'] = tf.Variable(tf.truncated_normal((self.hidden_size, self.hidden_size), stddev=0.01, seed=SEED), 
                                         dtype=tf.float32, name='Whh')
        all_weights['bhh'] = tf.Variable(tf.zeros(self.hidden_size), dtype=tf.float32, name='bhh')
        all_weights['Why'] = tf.Variable(tf.truncated_normal((self.hidden_size, self.vocab_size), stddev=0.01, seed=SEED), 
                                         dtype=tf.float32, name='Why')
        all_weights['bhy'] = tf.Variable(tf.zeros(self.vocab_size), dtype=tf.float32, name='bhy')
        
        return all_weights
    
    def train(self, X, Y, n_epoch, valid_char):
        epoch_disp = 2
        for epoch in range(n_epoch):
            for batch in range(len(X)):
                start_idx = batch
                end_idx = min(start_idx + self.truncated_backprop_length, len(x))
                if (end_idx - start_idx) < self.truncated_backprop_length :
                    break
                batchX = X[start_idx:end_idx].T
                batchY = Y[start_idx:end_idx].T
                loss, _ = self.sess.run((self.loss, self.optimizer), feed_dict={self.x: batchX, self.y: batchY})
            
            if(epoch %(epoch_disp)**2 ==0):
                print('Loss epoch({:})={:3.4f}'.format(epoch,loss))
                input_char = valid_char
                print([input_char], end=' ')
                batchX = np.array([self.vocab_onehot_dict[self.vocab_reverse_dict[i]].tolist() for i in input_char])
                self.generate_sequence(batchX)
                epoch_disp += 2
            
    def test(self, input_char):
        print([input_char], end=' ')
        batchX = np.array([self.vocab_onehot_dict[self.vocab_reverse_dict[i]].tolist() for i in input_char])
        self.generate_sequence(batchX, 55)
    
    def generate_sequence(self, X, n_samples=50):
        for i in range(n_samples):
            pred_series = self.sess.run(self.pred_series, feed_dict={self.x: X.T})
            pred_idx = np.argmax(pred_series, axis=2).flatten()
            X = np.roll(X, -1, axis=0)
            X[self.truncated_backprop_length-1] = self.vocab_onehot_dict[pred_idx[self.truncated_backprop_length-1]]
            print(self.vocab_dict[pred_idx[self.truncated_backprop_length-1]], end='')
        print('\n')
    
    def save_model(self, model_path):
        saver = tf.train.Saver()
        save_path = saver.save(self.sess, model_path)
        print("Model saved in file: %s" % save_path)


>The network is trained below, and during testing, the output which gives the max probabilty is feed back to the input to generate the next character and then its repeated recursively.

In [5]:
# training the Session
tf.reset_default_graph()
n_hidden = 50
n_vocab_size = len(vocab)
n_state_size = 20

rnn = charRNN(n_hidden, n_vocab_size, n_state_size, vocab_dict, vocab_reverse_dict, vocab_onehot_dict)


rnn.train(x, y, 100, 'Bob is a singer, he ')

save_path = './char_rnn'
rnn.save_model(save_path)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Loss epoch(0)=2.7951
['Bob is a singer, he ']                                                   

Loss epoch(16)=0.6482
['Bob is a singer, he '] sings well. Julia is also a singer, she sings well

Loss epoch(36)=0.1379
['Bob is a singer, he '] sings better also Julia is ie se Julia is also a s

Loss epoch(64)=0.0767
['Bob is a singer, he '] sings well. Julia is also a singer, she sings bett

Model saved in file: ./char_rnn


 > The training starts with bad predictions but as time progresses it prediction becomes better

In [6]:
# testing on a input
rnn.test('Bob is a singer, he ')


['Bob is a singer, he '] sings well. Julia is also a singer, she sings better a 



# Experiments with state variables
> The hidden state $h^{(t)} = g(W_{xh}*x^{(t)} + W_{hh}*h^{(t-1)} + b_{hh}) $  can be rewritten as $h^{(t)} = g(W_{h} * [\begin{array}{cccc} x^{(t)} \\ h^{(t-1)} \end{array}] + b_{hh}) $. The state equation can be simplified to a single weight with the concatenation of the previous state and the input as the actual input to a feed forward layer.
<img src='assets/input_concat.png'>
> The state variables encode information of the inputs the network has seen before.
> We will reset some of the values to zero, to see how it affects the network

In [7]:
# Load the model 

def pred_next_char(input_char, sess, state, weights, mask_state_var):
    X = np.array(vocab_onehot_dict[vocab_reverse_dict[input_char]].tolist())
    X = tf.constant(X, dtype=tf.float32)
    X = tf.reshape(X, [-1, n_vocab_size])
    mask = tf.constant(mask_state_var, dtype=tf.float32)
    state = tf.multiply(state, mask)
    h = tf.tanh(tf.matmul(X, weights['Wxh']) + tf.matmul(state, weights['Whh']) + weights['bhh'])
    p = tf.matmul(h, weights['Why'])+weights['bhy']
    out = sess.run(p)
    return out, h

def predict_var_char(input_char, mask_state_var):
    tf.reset_default_graph()
    loaded_graph = tf.Graph()
    with tf.Session(graph=loaded_graph) as sess:
        # Load model
        loader = tf.train.import_meta_graph(save_path+ '.meta')
        loader.restore(sess, save_path)
        loaded_weights = dict()
        loaded_init_state = loaded_graph.get_tensor_by_name('init_state:0')
        loaded_weights['Wxh'] = loaded_graph.get_tensor_by_name('Wxh:0')
        loaded_weights['Whh'] = loaded_graph.get_tensor_by_name('Whh:0')
        loaded_weights['bhh'] = loaded_graph.get_tensor_by_name('bhh:0')
        loaded_weights['Why'] = loaded_graph.get_tensor_by_name('Why:0')
        loaded_weights['bhy'] = loaded_graph.get_tensor_by_name('bhy:0')
    
        #input_char = 'do'
        print('In:', [input_char], end=' ')
        print('Out: ', end='')
        
        #h = tf.Variable(tf.zeros([1, n_hidden]), dtype=tf.float32)
        #new_var = tf.variables_initializer([h])
        #sess.run(new_var)
        
        h = loaded_init_state
        
        for i in range(55):
            for ch in input_char:
                out, h = pred_next_char(ch, sess, h, loaded_weights, mask_state_var)  
            idx_o = np.argmax(out, axis=1)
            input_char = vocab_dict[idx_o[0]]
            print(input_char, end='')
        print('\n')

> Testing whether the loaded model works well

In [8]:
predict_var_char('Bob is a',[1 for i in range(n_hidden)])


INFO:tensorflow:Restoring parameters from ./char_rnn
In: ['Bob is a'] Out:  singer, he sings well. Julia is also a singer, she sin



> Setting some random values of the state variable as zero, to notice the effect it has on the prediction


In [9]:
mask = [1 for i in range(n_hidden)]
#for i in range(n_hidden):
mask[7] = 0 # The first hidden unit is reset to zero
predict_var_char('Bob is a ',mask)


INFO:tensorflow:Restoring parameters from ./char_rnn
In: ['Bob is a '] Out: singer, she sings better a singer, she sings better a s



> The above example sets a value the state variable to zero, we see that the he/she prediction is lost by the network. So we can say that this value had the info about when to switch to he/she based on the context. This operation can be thought of as a forget gate like in LSTM where the he info is removed.

-- EOF --